In [1]:
import torch
from oa.model import OAPaiNN

/home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
model = OAPaiNN()

In [3]:
s0 = torch.rand(9, 8, dtype=torch.float32)

In [4]:
from torch_geometric.nn import radius_graph
pos = torch.tensor([[ 0.0072, -0.5687,  0.0000],
        [-1.2854,  0.2499,  0.0000],
        [ 1.1304,  0.3147,  0.0000],
        [ 0.0392, -1.1972,  0.8900],
        [ 0.0392, -1.1972, -0.8900],
        [-1.3175,  0.8784,  0.8900],
        [-1.3175,  0.8784, -0.8900],
        [-2.1422, -0.4239,  0.0000],
        [ 1.9857, -0.1365,  0.0000]], dtype = torch.float)

edge_index = radius_graph(pos, r=1.70, batch=None, loop=False)
print(edge_index)

tensor([[1, 2, 3, 4, 0, 5, 6, 7, 0, 8, 0, 0, 1, 1, 1, 2],
        [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 3, 4, 5, 6, 7, 8]])


In [5]:
subgraph_mask = torch.tensor([1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0], dtype=torch.long)[:, None]
out = model(s0, pos, edge_index, subgraph_mask=subgraph_mask)

In [6]:
from e3nn import o3
rot = o3.rand_matrix()
pos_rot = pos @ rot
edge_index_rot = radius_graph(pos_rot, r=1.70, batch=None, loop=False)
out_rot = model(s0, pos_rot, edge_index_rot, subgraph_mask=subgraph_mask)

In [7]:
(out[1] @ rot - out_rot[1]).max()

tensor(2.6226e-06, grad_fn=<MaxBackward1>)

In [8]:
(out[0] - out_rot[0]).max()

tensor(5.1260e-06, grad_fn=<MaxBackward1>)

In [9]:
from e3nn import o3
rot = o3.rand_matrix()
pos_rot = pos
pos_rot[:4] = pos[:4] @ rot
out_rot = model(s0, pos_rot, edge_index, subgraph_mask=subgraph_mask)
out[1][:4] = out[1][:4] @ rot

In [25]:
(out[0] - out_rot[0]).max()

tensor(3.6955e-06, grad_fn=<MaxBackward1>)

In [11]:
(out[1] - out_rot[1]).max()

tensor(3.0994e-06, grad_fn=<MaxBackward1>)